In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
movies_df=pd.read_csv("C:/Users/nikhil/Desktop/ml-latest-small/movies.csv",usecols=['movieId','title'])
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [9]:
ratings_df=pd.read_csv("C:/Users/nikhil/Desktop/ml-latest-small/ratings.csv",usecols=['movieId','userId','rating'])
ratings_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [11]:
df=pd.merge(movies_df,ratings_df,on='movieId')
df.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [12]:
df.shape

(100836, 4)

In [22]:
movie_combine_rating=df.dropna(axis=0,subset=['title'])
movie_combine_rating.head()
movie_rating_count=movie_combine_rating.groupby('title')['rating'].count().reset_index().rename(columns={'rating':'total_rating_count'})

In [23]:
movie_combine_rating.head()


,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [24]:
movie_rating_count.head()

,title,total_rating_count
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [27]:
rating_with_totalratingCount=movie_combine_rating.merge(movie_rating_count,left_on='title',right_on='title',how='left')

In [28]:
rating_with_totalratingCount.head()

,movieId,title,userId,rating,total_rating_count
0,1,Toy Story (1995),1,4.0,215
1,1,Toy Story (1995),5,4.0,215
2,1,Toy Story (1995),7,4.5,215
3,1,Toy Story (1995),15,2.5,215
4,1,Toy Story (1995),17,4.5,215


In [31]:
print(movie_rating_count['total_rating_count'].describe())

count    9719.000000
mean       10.375141
std        22.406220
min         1.000000
25%         1.000000
50%         3.000000
75%         9.000000
max       329.000000
Name: total_rating_count, dtype: float64


In [33]:
popularity_threshold=50
popular_movies_rating=rating_with_totalratingCount.query('total_rating_count>=@popularity_threshold')
popular_movies_rating.head()

,movieId,title,userId,rating,total_rating_count
0,1,Toy Story (1995),1,4.0,215
1,1,Toy Story (1995),5,4.0,215
2,1,Toy Story (1995),7,4.5,215
3,1,Toy Story (1995),15,2.5,215
4,1,Toy Story (1995),17,4.5,215


In [34]:
popular_movies_rating.shape

(41362, 5)

## creating a pivot index

In [36]:
movie_matrix_df=popular_movies_rating.pivot_table(index='title',columns='userId',values='rating').fillna(0)
movie_matrix_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0


### creating array matrix:

In [37]:
from scipy.sparse import csr_matrix

In [41]:
movie_matrix=csr_matrix(movie_matrix_df.values)

In [44]:
from sklearn.neighbors import NearestNeighbors

In [45]:
model_knn=NearestNeighbors(metric='cosine',algorithm='brute')

In [46]:
model_knn.fit(movie_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [47]:
movie_matrix_df.shape

(450, 606)

In [49]:
query_index=np.random.choice(movie_matrix_df.shape[0])
print(query_index)

73


In [50]:
distances,indices=model_knn.kneighbors(movie_matrix_df.iloc[query_index,:].values.reshape(1,-1),n_neighbors=6)

In [66]:
for i in range(0,len(distances.flatten())):
    if i == 0:
        print("REcommendation for {0}:\n".format(movie_matrix_df.index[query_index]))
    else:
        print("{0}:{1},with distance of {2}".format(i,movie_matrix_df.index[indices.flatten()[i]],distances.flatten()[i]))
    
        

REcommendation for Bridget Jones's Diary (2001):

1:Moulin Rouge (2001),with distance of 0.4403067942812403
2:Love Actually (2003),with distance of 0.4454071035192032
3:Notting Hill (1999),with distance of 0.47872315893217277
4:Legally Blonde (2001),with distance of 0.48277962485401527
5:What Women Want (2000),with distance of 0.5062064233135464


In [65]:
indices.flatten()[i]

434